기준금리 크롤링

In [6]:
import requests
import datetime
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

base_url= "https://www.bok.or.kr/portal/singl/baseRate/list.do?dataSeCd=01&menuNo=200643"
response = requests.get(base_url, 'html.parser')
soup= BeautifulSoup(response.text, 'lxml')

In [7]:
dict_list= []
# 53
for i in range(53):
    dict_list.append({
        'date': pd.to_datetime(datetime.datetime.strptime(soup.find_all('td')[3*i].text + soup.find_all('td')[3*i+1].text, '%Y%m월 %d일')),
        'RATE': float(soup.find_all('td')[3*i+2].text)
    })

In [8]:
pd_result= pd.DataFrame(dict_list)

In [9]:
pd_result

,date,RATE
0,2023-01-13,3.50
1,2022-11-24,3.25
2,2022-10-12,3.00
3,2022-08-25,2.50
4,2022-07-13,2.25
5,2022-05-26,1.75
6,2022-04-14,1.50
7,2022-01-14,1.25
8,2021-11-25,1.00
9,2021-08-26,0.75


In [10]:
# date range 일별 추가
idx= pd.date_range(pd_result.date.min(), pd_result.date.max())


In [ ]:
idx= pd.Series(idx)

In [12]:
a=pd.DataFrame({'data':idx[~idx.isin(pd_result.date)],'RATE':np.nan})

In [24]:
b=pd.concat([a,pd_result])

In [25]:
b_1=b.sort_values('date')

In [26]:
c=b_1.reset_index(drop=True)

In [32]:
d=c.ffill(axis=0).reset_index(drop=True)
d

,data,RATE,date
0,NaT,5.25,2000-10-05
1,NaT,5.00,2001-02-08
2,NaT,4.75,2001-07-05
3,NaT,4.50,2001-08-09
4,NaT,4.00,2001-09-19
...,...,...,...
8131,2023-01-08,3.50,2023-01-13
8132,2023-01-09,3.50,2023-01-13
8133,2023-01-10,3.50,2023-01-13
8134,2023-01-11,3.50,2023-01-13


In [30]:
# 없는 행에 Nan을 넣어 병합한 후 Nan값을 이전 값으로 메꾸기
baserate= pd.concat([pd.DataFrame({'date': idx[~idx.isin(pd_result.date)], 'RATE': np.nan}),pd_result]).sort_values('date').reset_index(drop=True).ffill(axis=0)

In [34]:
baserate[7000:7500]

,date,RATE
7000,2019-12-05,1.25
7001,2019-12-06,1.25
7002,2019-12-07,1.25
7003,2019-12-08,1.25
7004,2019-12-09,1.25
...,...,...
7495,2021-04-13,0.50
7496,2021-04-14,0.50
7497,2021-04-15,0.50
7498,2021-04-16,0.50


In [55]:
import pandas as pd
from datetime import datetime

interest=pd.read_csv('../interest_rate.csv',index_col=None)
interest['date'] = pd.to_datetime(interest['date'])
interest=interest.iloc[:,2:]


In [52]:
interest['u_d'].unique()

array([ 0.  ,  0.25, -0.25, -0.5 ,  0.5 ,   nan])

In [56]:
interest = interest.merge(interest.assign(date = interest.date+pd.Timedelta(days=30)), 
               on='date',
               how='left', suffixes=['', '_30days'])


In [57]:
interest

,date,i_r,after_date,i_r+30,u_d,i_r_30days,after_date_30days,i_r+30_30days,u_d_30days
0,2011-03-10,3.0,2011.4.9,3.0,0.0,NaN,NaN,NaN,NaN
1,2011-03-11,3.0,2011.4.10,3.0,0.0,NaN,NaN,NaN,NaN
2,2011-03-12,3.0,2011.4.11,3.0,0.0,NaN,NaN,NaN,NaN
3,2011-03-13,3.0,2011.4.12,3.0,0.0,NaN,NaN,NaN,NaN
4,2011-03-14,3.0,2011.4.13,3.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4230,2022-10-08,2.5,2022.11.7,NaN,NaN,2.5,2022.10.8,2.5,0.0
4231,2022-10-09,2.5,2022.11.8,NaN,NaN,2.5,2022.10.9,2.5,0.0
4232,2022-10-10,2.5,2022.11.9,NaN,NaN,2.5,2022.10.10,2.5,0.0
4233,2022-10-11,2.5,2022.11.10,NaN,NaN,2.5,2022.10.11,3.0,0.5


In [58]:
baserate.dropna(inplace=True)

In [60]:
baserate = baserate.merge(baserate.assign(date = baserate.date+pd.Timedelta(days=30)), 
               on='date',
               how='left', suffixes=['', '_30days'])


In [61]:
baserate

,date,RATE,RATE_30days
0,2000-10-05,5.25,NaN
1,2000-10-06,5.25,NaN
2,2000-10-07,5.25,NaN
3,2000-10-08,5.25,NaN
4,2000-10-09,5.25,NaN
...,...,...,...
8131,2023-01-09,3.25,3.25
8132,2023-01-10,3.25,3.25
8133,2023-01-11,3.25,3.25
8134,2023-01-12,3.25,3.25


In [62]:
baserate['change'] = np.select(
    condlist=[
        (baserate.RATE > baserate.RATE_30days), 
        (baserate.RATE == baserate.RATE_30days),
    ], 
    choicelist=[
        1,
        0,
    ],
    default=-1
)

In [63]:
baserate

,date,RATE,RATE_30days,change
0,2000-10-05,5.25,NaN,-1
1,2000-10-06,5.25,NaN,-1
2,2000-10-07,5.25,NaN,-1
3,2000-10-08,5.25,NaN,-1
4,2000-10-09,5.25,NaN,-1
...,...,...,...,...
8131,2023-01-09,3.25,3.25,0
8132,2023-01-10,3.25,3.25,0
8133,2023-01-11,3.25,3.25,0
8134,2023-01-12,3.25,3.25,0


In [64]:
baserate.to_csv('../baserate.csv',index=False)

콜금리

In [ ]:
call_url= "https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL&page={}"
page_num= 622 # 2005년 5월~

In [ ]:
call_list= []

for i in range(1,page_num+1):
    call_list.append(BeautifulSoup(requests.get(call_url.format(i), 'html.parser').text,'lxml').find_all('td'))

In [ ]:
result_list= []

for list in call_list:
    
    for i in range(len(list)//4):
        result_list.append({
            'date': list[4*i].text.strip(),
            'RATE': float(list[4*i+1].text)
        })

In [ ]:
result_list

In [ ]:
pd_result2= pd.DataFrame(result_list)
pd_result2.date= pd_result2.date.astype('datetime64')

In [ ]:
# 원하는 전체 date range
idx2= pd.date_range(pd_result2.date.min(), pd_result2.date.max())
idx2= pd.Series(idx2)

In [ ]:
# 없는 행에 Nan을 넣어 병합한 후 Nan값을 이전 값으로 메꾸기
callrate= pd.concat([pd.DataFrame({'date': idx2[~idx2.isin(pd_result2.date)], 'RATE': np.nan}),pd_result2]).sort_values('date').reset_index(drop=True).ffill(axis=0)

In [ ]:
callrate

In [ ]:
callrate = callrate.merge(callrate.assign(date = callrate.date+pd.Timedelta(days=31)), 
               on='date',
               how='left', suffixes=['', '_30days_ago'])


In [ ]:
callrate.dropna(inplace=True)

In [ ]:
callrate

In [ ]:
def calc_rate_change(df, date):
    df['rate_change'] = df['RATE'] - df['RATE_30days_ago']
    df['rate_change'].loc[df['polarity'] >= 1.1] = 1
    df['rate_change'].loc[df['polarity'] < 0] = -1
    df['rate_change'].loc[0 <= df['polarity'] < 1.1] = 0
    return df

In [ ]:
# calc_rate_change(callrate)

In [ ]:
callrate['change'] = np.select(
    condlist=[
        (callrate.RATE > callrate.RATE_30days_ago), 
        (callrate.RATE == callrate.RATE_30days_ago),
    ], 
    choicelist=[
        'up',
        'same',
    ],
    default='down'
)

In [ ]:
callrate['change'] = np.select(
    condlist=[
        ((callrate.RATE - callrate.RATE_30days_ago) >= 1.1), 
        (0 <= (callrate.RATE - callrate.RATE_30days_ago) < 1.1),
    ], 
    choicelist=[
        'up',
        'same',
    ],
    default='down'
)

In [ ]:
callrate

In [ ]:
callrate.to_csv('./callrate.csv',  index=False)